---
# LLM based Insight & Personalsed Content Generator

This notebook describes the architecture for a model that generates property content tailored to different target audiences, e.g. (families, expats, students, investors) for use by product and marketing teams


---
# Proposed System Architecture

Given differences between information that would be relevant for sales vs rentals, I would use a core central platform with listing-type-specific configuration
* shared RAG + orchestration
* listing-type-specific prompt templates, metrics, and guardrails

<img src="images/LLM_shared_platform.png" width="1000">


## High level flow

1. Input
    * property metadata (structured)
    * lusting type (rental/sale)
    * target audience
    * language/tone
2. Retrieval (RAG)
    * Fetch neighborhood facts (schools, safety, public transport, amenities)
    * Market context (rent index, vacancy, mortgage rates, price trends)
    * POI distances (e.g supermarket) / commute times
    * Internal policies (allowed claims, disclaimers, phrasing preferences, evidence requirements)
3. Fact building & configuration
    * Compute derived metrics (e.g. rent/m² or price/m² depending on listing_type)
    * Load listing-type-specific constraints (allowed financial fields, required disclaimers)
4. Content planning (LLM)
    * Extract what matters for each audience (e.g., schools for families, yield for investors)
    * Define structured content plan
5. Content generation (LLM)
    * Generate description strictly from a provided grounding bundle
    * Output includes fact references (citations to retrieved snippets/fields)
6. Validation & guardrails (Rule based, schema validator, LLM)
    * Claim checker: ensure every numeric/location/superlative claim is supported
    * Schema validator: ensure required sections and constraints
    * Policy filter: block disallowed claims (“safest area”, “guaranteed ROI”) unless supported + allowed
7. Revision (Loop back to LLM)
    * If validation fails, return to LLM Writer
8. Output
    * Multiple descriptions
    * Structured fact table used
    * Confidence flags

## Prompting Strategy

Pipeline:
1. Assemble facts block (property metadata + retrieved snippets + derived metrics)
2. Planner prompt consumes that facts block + target audience + listing type
3. Planner returns a JSON plan:
    * chosen facts (with IDs)
    * outline/sections
    * key messages per audience
    * banned claims + required disclaimers
    * style constraints
4. Writer prompt receives:
    * the same facts block
    * the planner JSON
    * formatting rules + JSON output schema
5. Writer generates final description + claim/source mapping
6. Validator checks claims against facts_used and retrieved snippet IDs.

---
# Data Sources

## Internal listing data

Source: internal databases

Example fields:
* location: city, neighborhood, coordinates
* property type, year built, condition/renovations
* size (m²), rooms, floor, lift, balcony
* rent/price, oncosts, parking
* energy rating / heating type (if available)
* pet policy
* furnished/unfurnished
* availability date
* photos

## Neighbourhood & Points of interest facts

Potentital Sources:
* public transport: https://data.opentransportdata.swiss/en/dataset/traffic-points-full
* public transport: https://registry.opendata.aws/schweizer-haltestellen-oev/?utm_source=chatgpt.com
* poi: open street map api: https://www.openstreetmap.org/export

Example fields
* nearby schools/daycare count + distances
* grocery, parks, gyms, medical, nightlife
* public transport stops + frequency proxy
* safety proxy (e.g., “crime incidents per 1,000 residents” as a synthetic stat)
* “quietness index” / “green space index” (synthetic)

## Market data

Potential sources:
* Federal statistic office
* Internal databases

Example reports
* rental price index YoY and QoQ
* vacancy rate
* mortgage rate
* average time-on-market
* comparable listing ranges (by m² / n rooms)

## Macroeconomic indicators

Potential sources:
* Federal statistic office

Example indicators
* inflation rate
* unemployment

---
# Hallucination Approach 

## Grounding rules

The LLM only gets given:
* property metadata
* retrieved neighbourhood/market snippets
* allowed/dissallowed claims policy (e.g. dissallow "guaranteed returns")

Citations are required for
* numbers (rent index %, distances, commute times)
* comparative phrases ("above average", "high demand")

## Post generation validation

Automated checks will be implemented
* Claim-to-source linking
    * each claim has source_id referencing a field/snippet
* Numeric consistency checks
    * rent = metadata rent
    * computed CHF/m² = rent/m² (within tolerance)
* Entity checks
    * neighbourhood name matches metadata
    * POIs referenced exist in retrieved POI list
* Style constraints
    * avoid forbidden phrases
    * length bounds per audience (e.g., students shorter)

## Human-in-the-loop

Anything flagged low confidence goes to review

Review UI shows
* generated text
* extracted claims
* their sources
* quick approve/edit

---
# Evaluation

## Automated evaluation

Factuality / grounding
* % of claims with valid source links
* contradiction rate vs metadata

Audience fit
* A lightweight classifier or LLM-judge will evaluate the text and classify which audience is best fit, e.g it would likely look at wording like this
    * families: schools/parks/safety/space
    * expats: commute/international vibe/furnished/services
    * students: transit to campus/budget/roommates
    * investors: yield/demand/liquidity risks

Readability
* length
* sentence complexity
* banned terms

Diversity of descriptions (if a concern)
* n-gram overlap across the different audience outputs

## Human evaluation

Marketing/product team would be used to review the outputs before final acceptance. A weekly sample might also be made

Rate 1–5
* audience alignment
* clarity
* compliance risk
* would you publish this?

## Test set
* 30–50 properties across neighborhoods and price tiers
* Include edge cases / conflicting data:
    * missing data
    * studio near nightlife
    * family home but no nearby schools
    * investor property with high rent but high vacancy area

# Prototype

In [1]:
import os
import json
import re
from openai import OpenAI
import logging

# logger
logger = logging.getLogger()
handler = logging.StreamHandler()
handler.name = 'handler'
formatter = logging.Formatter(fmt='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-5-mini"

costs = {
    "gpt-5-mini": {'input_token': 0.025, 'output_token': 2},
    "gpt-5"     : {'input_token': 0.125, 'output_token': 10}
}

def llm_cost_calculation(usage_details, model):
    model_cost = costs[model]
    
    input_tokens = usage_details.input_tokens
    output_tokens = usage_details.output_tokens
    total_tokens = usage_details.total_tokens

    input_cost = input_tokens / 1_000_000 * model_cost["input_token"]
    output_cost = output_tokens / 1_000_000 * model_cost["output_token"]
    total_cost = round(input_cost + output_cost, 2)
    
    return {
        "model": model,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": total_tokens,
        "cost": total_cost,
    }

## Simulated Property + Retrieval (RAG)

In [2]:
property_data = {
    "listing_type": "rental",
    "city": "Zurich",
    "neighborhood": "Wiedikon",
    "rooms": 3.5,
    "size_m2": 82,
    "balcony_m2": 7,
    "year_built": 2016,
    "elevator": True,
    "heating": "Underfloor heating",
    "rent_chf": 3150,
    "rent_per_m2": 38.4,
    "utilities_chf": 250,
    "parking_chf": 180,
    "available_from": "2026-04-01"
}

retrieved_docs = [
    {"id": "S1", "text": "Primary schools within 1.2 km: 4. Parks within 10-minute walk: 3."},
    {"id": "S2", "text": "Nearest S-Bahn station within 650 m. Peak frequency approx every 7-10 minutes."},
    {"id": "M1", "text": "District-level vacancy estimate: 0.7%. Zurich rental index YoY +2.1%."},
    {"id": "M2", "text": "Local median advertised rent: CHF 37/m²."}
]

def build_facts_block(property_data, retrieved_docs):
    return {
        "meta": property_data,
        "retrieved": retrieved_docs
    }

facts_block = build_facts_block(property_data, retrieved_docs)
facts_block

{'meta': {'listing_type': 'rental',
  'city': 'Zurich',
  'neighborhood': 'Wiedikon',
  'rooms': 3.5,
  'size_m2': 82,
  'balcony_m2': 7,
  'year_built': 2016,
  'elevator': True,
  'heating': 'Underfloor heating',
  'rent_chf': 3150,
  'rent_per_m2': 38.4,
  'utilities_chf': 250,
  'parking_chf': 180,
  'available_from': '2026-04-01'},
 'retrieved': [{'id': 'S1',
   'text': 'Primary schools within 1.2 km: 4. Parks within 10-minute walk: 3.'},
  {'id': 'S2',
   'text': 'Nearest S-Bahn station within 650 m. Peak frequency approx every 7-10 minutes.'},
  {'id': 'M1',
   'text': 'District-level vacancy estimate: 0.7%. Zurich rental index YoY +2.1%.'},
  {'id': 'M2', 'text': 'Local median advertised rent: CHF 37/m².'}]}

# Prompt & Schema Store

In [3]:
def build_planner_prompt(audience, listing_type, facts_block, tone='professional'):

    system_prompt = f"""
    You are a content planner for real-estate listings. Your job is NOT to write marketing copy.
    You must produce a structured content plan that selects which facts may be used and how they should be presented.
    You must follow the rules:
    - Use ONLY information explicitly present in the FACTS block.
    - If a desired fact is missing, do not invent it.
    - Avoid prohibited claim types (see POLICY).
    - Return valid JSON only.
    """

    user_prompt = f"""
    TASK
    Create a structured content plan for a real-estate listing description tailored to:
    - Audience: {audience}
    - Listing type: {listing_type}
    - Desired tone: {tone}
    - Language: English

    GOALS
    1) Select the most relevant facts for this audience and listing type.
    2) Produce an outline with intent for each section.
    3) Specify hard constraints for the writer: what must be cited, what must be avoided, and required disclaimers.
    4) Set length and formatting targets.

    POLICY (Prohibited / High-risk claims)
    - No guarantees (e.g., guaranteed ROI, will increase in value, risk-free)
    - No safety superlatives (e.g., safest, no crime, perfectly safe)
    - No school quality claims unless explicitly supported by a retrieved snippet (e.g., official rating)
    - No invented POIs or commute times not present in FACTS
    - Avoid subjective absolutes (e.g., best, most desirable) unless supported by retrieved ranking data

    FACTS BLOCK
    {facts_block}

    Return only JSON.
    """

    return system_prompt, user_prompt
    

def build_writer_prompt(audience, listing_type, facts_block, planner_output):

    system_prompt = f"""
    You are a real-estate copywriter. Your task is to generate marketing copy that is strictly grounded in approved facts.
    
    HARD CONSTRAINTS:
    - You may ONLY use facts explicitly listed in plan.facts_to_use.
    - If a fact is not listed in plan.facts_to_use, you must not reference it.
    - You must not introduce any new numbers, dates, distances, percentages, rankings, or named POIs.
    - Every numeric or measurable claim (CHF, %, m², km, m, minutes, dates) MUST appear in claims[] with a valid source_id.
    - The source_id must match an ID present in plan.facts_to_use.
    - Do not infer, estimate, or extrapolate values.
    - Output must be valid JSON matching the required schema.
    """

    user_prompt = f"""
    TASK
    Write a tailored listing description for:
    - Audience: {audience}
    - Listing type: {listing_type}
    - Language: English

    INPUTS
    1) FACTS BLOCK (for reference, but only allowed facts may be used):
    {facts_block}

    2) PLAN (authoritative source of allowed facts and constraints):
    {planner_output}

    STYLE REQUIREMENTS (mandatory):
    - Follow plan.style.tone.
    - Keep total word count within plan.style.length_words range.
    - Follow the exact section order defined in plan.outline.
    - Use short paragraphs.
    - Include a bullet list of highlights (3-5 bullets).

    OUTPUT REQUIREMENTS:
    Return JSON with:
    - title (string)
    - description (string)
    - highlights (array of strings)
    - claims (array of objects with claim_text and source_id)
    - facts_used (array of source_ids)

    Return only JSON.
    """

    return system_prompt, user_prompt

## Planner Step

In [4]:
def run_planner(listing_type, audience, facts_block):


    with open('data/planner_schema.json') as f:
        planner_schema = json.load(f)

    # insert valid ids into schema
    valid_ids = (
        [f"meta.{k}" for k in facts_block["meta"].keys()] +
        [doc["id"] for doc in facts_block["retrieved"]]
    )
    planner_schema["properties"]["facts_to_use"]["items"]["enum"] = valid_ids

    system_prompt, user_prompt = build_planner_prompt(audience, listing_type, facts_block, tone='professional')

    input_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = client.responses.create(
        model=MODEL,
        input=input_messages,
        timeout=300, #seconds
        text={
            "format": {
                "type": "json_schema",
                "schema": planner_schema,
                "name": "planner_output",
                "strict": True
            }
        }
    )
    
    return response

In [5]:
def run_planner(listing_type, audience, facts_block):


    with open('data/planner_schema.json') as f:
        planner_schema = json.load(f)

    # insert valid ids into schema
    valid_ids = (
        [f"meta.{k}" for k in facts_block["meta"].keys()] +
        [doc["id"] for doc in facts_block["retrieved"]]
    )
    planner_schema["properties"]["facts_to_use"]["items"]["enum"] = valid_ids

    system_prompt, user_prompt = build_planner_prompt(audience, listing_type, facts_block, tone='professional')

    input_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = client.responses.create(
        model=MODEL,
        input=input_messages,
        timeout=300, #seconds
        text={
            "format": {
                "type": "json_schema",
                "schema": planner_schema,
                "name": "planner_output",
                "strict": True
            }
        }
    )
    usage = response.usage
    cost_info = llm_cost_calculation(usage, MODEL)

    planner_output = json.loads(response.output[1].content[0].text)
    return planner_output, cost_info

## Writer Step

In [6]:
def run_writer(planner_output, facts_block):

    with open('data/writer_schema.json') as f:
        writer_schema = json.load(f)

    system_prompt, user_prompt = build_writer_prompt(audience, listing_type, facts_block, planner_output)
    input_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = client.responses.create(
        model=MODEL,
        input=input_messages,
        timeout=300, #seconds
        text={
            "format": {
                "type": "json_schema",
                "schema": writer_schema,
                "name": "planner_output",
                "strict": True
            }
        }
    )
    usage = response.usage
    cost_info = llm_cost_calculation(usage, MODEL)

    writer_output = json.loads(response.output[1].content[0].text)
    return writer_output, cost_info

In [10]:
test = True

listing_type = "rental"
audience = "families"

if test:
    response = run_planner(listing_type, audience, facts_block)
    planner_output, cost_info = run_planner(listing_type, audience, facts_block)
    print("\n" + "="*80)
    print(cost_info)
    display(planner_output)
    writer_output, cost_info = run_writer(planner_output, facts_block)
    print("\n" + "="*80)
    print(cost_info)
    display(writer_output)

03/01/2026 05:54:16 PM HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
03/01/2026 05:54:48 PM HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"



{'model': 'gpt-5-mini', 'input_tokens': 900, 'output_tokens': 2406, 'total_tokens': 3306, 'cost': 0.0}


{'audience': 'families',
 'listing_type': 'rental',
 'positioning': {'primary_angle': 'A modern, family-oriented 3.5-room apartment in Wiedikon (Zurich) with nearby schools, parks and frequent public transport.',
  'secondary_angle': 'Transparent pricing and local market context to help families compare value and availability.'},
 'outline': [{'section': 'Headline / Lead',
   'goal': "One-sentence summary stating apartment type, neighbourhood and move-in date to capture family renters' attention."},
  {'section': 'Key apartment facts (quick bullets)',
   'goal': 'Present essential facts at a glance: rooms, size, balcony, floor-level features (elevator), heating, year built, rent and additional charges.'},
  {'section': 'Family-focused features',
   'goal': 'Highlight features important to families: room layout (3.5 rooms), balcony size, underfloor heating, elevator and year built (modern construction) — prompt reader to imagine daily life.'},
  {'section': 'Neighborhood & local ameniti

03/01/2026 05:56:14 PM HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"



{'model': 'gpt-5-mini', 'input_tokens': 1820, 'output_tokens': 6116, 'total_tokens': 7936, 'cost': 0.01}


{'audience': 'families',
 'listing_type': 'rental',
 'title': 'Modern 3.5‑room family apartment in Wiedikon (Zurich) — available 2026-04-01',
 'description': 'Modern 3.5-room rental apartment in Wiedikon (Zurich), available from 2026-04-01.\n\nKey apartment facts:\n- 3.5 rooms, 82 m² living space\n- 7 m² balcony\n- Built in 2016; elevator in the building; underfloor heating\n- Monthly rent CHF 3,150; utilities CHF 250; parking CHF 180 (parking charged separately)\n\nThis modern 3.5-room layout offers practical living space and a balcony for outdoor time. The building’s 2016 construction, elevator and underfloor heating support an easy daily routine for families and make moving with children and strollers straightforward.\n\nNeighborhood and transport: there are 4 primary schools within 1.2 km and 3 parks within a 10-minute walk, supporting family life and outdoor play. The nearest S-Bahn station is within 650 m, with peak services running approximately every 7–10 minutes, providing fre

## Hallucination Validation & Evaluation

In [11]:
NUM_PATTERN = re.compile(
    r"(CHF\s?\d[\d,]*\.?\d*|\d+\.?\d*\s?%|\d+\.?\d*\s?m²|\d+\.?\d*\s?km|\d+\.?\d*\s?m|\d{4}-\d{2}-\d{2})"
)

def validate_output(writer_output, facts_block, planner_output=None):

    issues = []

    description = writer_output["description"]

    # All numeric mentions must be cited
    numeric_mentions = NUM_PATTERN.findall(description)
    cited_texts = [c["claim_text"] for c in writer_output["claims"]]

    for mention in numeric_mentions:
        if not any(mention in claim for claim in cited_texts):
            issues.append(f"Uncited numeric mention: {mention}")

    # Validate source ids exist
    valid_ids = (
        [f"meta.{k}" for k in facts_block["meta"].keys()] +
        [doc["id"] for doc in facts_block["retrieved"]]
    )

    snippet_map = {d["id"]: d["text"] for d in facts_block["retrieved"]}

    for claim in writer_output["claims"]:
        claim_text = claim["claim_text"]
        source_id = claim["source_id"]

        if source_id not in valid_ids:
            issues.append(f"Invalid source_id: {source_id}")
            continue

        # Meta check
        if source_id.startswith("meta."):
            field = source_id.split(".", 1)[1]
            actual_value = str(facts_block["meta"][field])

            if actual_value not in claim_text:
                issues.append(
                    f"Meta mismatch: '{claim_text}' does not contain '{actual_value}'"
                )

        # Snippet check
        else:
            snippet_text = snippet_map[source_id]

            if claim_text not in snippet_text:
                issues.append(
                    f"Claim '{claim_text}' not found in snippet {source_id}"
                )

    return issues

In [12]:
def compute_confidence(validation_issues):

    score = 1.0

    for issue in validation_issues:
        if "Invalid source_id" in issue:
            score -= 0.3
        elif "Uncited numeric mention" in issue:
            score -= 0.2
        elif "Meta mismatch" in issue:
            score -= 0.2
        elif "not found in snippet" in issue:
            score -= 0.15
        else:
            score -= 0.1

    return max(round(score,2), 0.0)

In [13]:
if test:
    validation_issues = validate_output(writer_output, facts_block)
    display(validation_issues)
    confidence_score = compute_confidence(validation_issues)
    display(confidence_score)

["Meta mismatch: 'Elevator present in the building' does not contain 'True'",
 "Meta mismatch: 'Monthly rent CHF 3,150' does not contain '3150'",
 "Claim '4 primary schools within 1.2 km' not found in snippet S1",
 "Claim '3 parks within a 10-minute walk' not found in snippet S1",
 "Claim 'Peak frequency approximately every 7-10 minutes' not found in snippet S2",
 "Claim 'District-level vacancy estimate 0.7%' not found in snippet M1",
 "Claim 'Zurich rental index year-on-year +2.1%' not found in snippet M1",
 "Claim 'Local median advertised rent CHF 37/m²' not found in snippet M2"]

0.0

# Generate Multiple Audiences

In [14]:
listing_type = "rental"
audiences = ["families", "expats", "students", "investors"]

results = {}
cost = 0

for audience in audiences:
    logger.info(f'Content Generation Started for: {audience}')
    planner_output, cost_info = run_planner(listing_type, audience, facts_block)
    cost += cost_info['cost']
    logger.info(f'{audience}: Planner completed')

    writer_output, cost_info = run_writer(planner_output, facts_block)
    cost += cost_info['cost']
    logger.info(f'{audience}: Writer completed')

    issues = validate_output(writer_output, facts_block)
    logger.info(f'{audience}: Validation completed')

    confidence_score = compute_confidence(validation_issues)
    logger.info(f'{audience}: Confidence score: {validation_issues}')

    results[audience] = {
        "planner": planner_output,
        "writer": writer_output,
        "validation_issues": issues,
        "validation_issues": validation_issues
    }

print(f'Total LLM cost {cost}')
results

03/01/2026 06:16:26 PM Content Generation Started for: families
03/01/2026 06:17:03 PM HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
03/01/2026 06:17:03 PM families: Planner completed
03/01/2026 06:19:05 PM HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
03/01/2026 06:19:05 PM families: Writer completed
03/01/2026 06:19:05 PM families: Validation completed
03/01/2026 06:19:05 PM families: Confidence score: ["Meta mismatch: 'Elevator present in the building' does not contain 'True'", "Meta mismatch: 'Monthly rent CHF 3,150' does not contain '3150'", "Claim '4 primary schools within 1.2 km' not found in snippet S1", "Claim '3 parks within a 10-minute walk' not found in snippet S1", "Claim 'Peak frequency approximately every 7-10 minutes' not found in snippet S2", "Claim 'District-level vacancy estimate 0.7%' not found in snippet M1", "Claim 'Zurich rental index year-on-year +2.1%' not found in snippet M1", "Claim 'Local median advertise

Total LLM cost 0.07


{'families': {'planner': {'audience': 'families',
   'listing_type': 'rental',
   'positioning': {'primary_angle': 'Family-friendly 3.5-room apartment in Wiedikon with nearby primary schools and parks — comfortable living space and balcony for family use.',
    'secondary_angle': 'Modern, low-maintenance building (2016) with elevator and underfloor heating; good public transport access and transparent cost breakdown.'},
   'outline': [{'section': 'Headline',
     'goal': 'One-line summary that highlights key selling points for families (rooms, neighborhood, availability, monthly rent).'},
    {'section': 'Quick facts (at-a-glance)',
     'goal': 'List essential, verifiable facts: type, city/neighborhood, rooms, size, balcony size, year built, elevator, heating, rent, utilities, parking fee, available from.'},
    {'section': 'Why it works for families',
     'goal': 'Explain family-relevant features using only supported facts: number of rooms and size, balcony, proximity to primary sch

In [15]:
for audience in audiences:
    print("\n" + "="*80)
    print(f'Audience: {audience}')
    print(f'Title: {results[audience]['writer']['title']}')
    print(f'Description:\n {results[audience]['writer']['description']}')
    


Audience: families
Title: Family-friendly 3.5-room rental in Wiedikon — available 2026-04-01 at CHF 3,150/month
Description:
 Headline
Family-friendly 3.5-room apartment in Wiedikon — available 2026-04-01 at CHF 3,150/month.

Quick facts (at-a-glance)
Type: Rental in Zurich, Wiedikon. 3.5 rooms, 82 m² living space, 7 m² balcony. Built 2016. Elevator present. Heating: Underfloor heating. Rent CHF 3,150 / month; utilities CHF 250; parking CHF 180. Available from 2026-04-01.

Why it works for families
The 3.5-room layout and 82 m² provide clear family living space, plus a 7 m² balcony for outdoor time. Primary schools within 1.2 km: 4. Parks within 10-minute walk: 3.

Practical features & comfort
The building was completed in 2016 and includes an elevator. The apartment benefits from underfloor heating for even warmth and a practical 3.5-room arrangement.

Transport & local access
Nearest S-Bahn station within 650 m. Peak frequency approx every 7-10 minutes.

Costs & availability
Monthly